# SQL Practice Exercises

Test your SQL skills with real-world problems!

## Difficulty Levels
🟢 Beginner | 🟡 Intermediate | 🔴 Advanced

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(':memory:')

## Setup: E-commerce Database

In [ ]:
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT,
    city TEXT,
    signup_date TEXT
)
''')

cursor.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date TEXT,
    total_amount REAL,
    status TEXT
)
''')

cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    category TEXT,
    price REAL
)
''')

cursor.execute('''
CREATE TABLE order_items (
    item_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER
)
''')

# Insert sample data
customers = [
    (1, 'Alice', 'alice@email.com', 
     'New York', '2023-01-15'),
    (2, 'Bob', 'bob@email.com', 
     'Los Angeles', '2023-02-20'),
    (3, 'Charlie', 'charlie@email.com', 
     'Chicago', '2023-03-10'),
    (4, 'David', 'david@email.com', 
     'New York', '2023-04-05'),
    (5, 'Eve', 'eve@email.com', 
     'Chicago', '2023-05-12')
]

orders = [
    (1, 1, '2023-06-01', 150.00, 'completed'),
    (2, 1, '2023-06-15', 200.00, 'completed'),
    (3, 2, '2023-06-10', 75.00, 'completed'),
    (4, 3, '2023-06-20', 300.00, 'pending'),
    (5, 4, '2023-06-25', 125.00, 'completed'),
    (6, 1, '2023-07-01', 180.00, 'cancelled'),
    (7, 5, '2023-07-05', 90.00, 'completed')
]

products = [
    (1, 'Laptop', 'Electronics', 1000.00),
    (2, 'Mouse', 'Electronics', 25.00),
    (3, 'Keyboard', 'Electronics', 75.00),
    (4, 'Desk', 'Furniture', 300.00),
    (5, 'Chair', 'Furniture', 200.00),
    (6, 'Monitor', 'Electronics', 400.00)
]

order_items = [
    (1, 1, 2, 2),  # Order 1: 2 Mice
    (2, 1, 3, 1),  # Order 1: 1 Keyboard
    (3, 2, 6, 1),  # Order 2: 1 Monitor
    (4, 3, 2, 3),  # Order 3: 3 Mice
    (5, 4, 4, 1),  # Order 4: 1 Desk
    (6, 5, 5, 1),  # Order 5: 1 Chair
    (7, 7, 3, 1),  # Order 7: 1 Keyboard
    (8, 7, 2, 1)   # Order 7: 1 Mouse
]

cursor.executemany(
    'INSERT INTO customers VALUES (?,?,?,?,?)', 
    customers
)
cursor.executemany(
    'INSERT INTO orders VALUES (?,?,?,?,?)', 
    orders
)
cursor.executemany(
    'INSERT INTO products VALUES (?,?,?,?)', 
    products
)
cursor.executemany(
    'INSERT INTO order_items VALUES (?,?,?,?)', 
    order_items
)
conn.commit()

print("Database created successfully!")

## 🟢 Beginner Exercises

### Exercise 1: List all customers from Chicago

In [ ]:
# Your solution here


In [ ]:
# Solution
query = '''
SELECT name, email, city
FROM customers
WHERE city = 'Chicago'
'''
pd.read_sql_query(query, conn)

### Exercise 2: Count total orders

In [ ]:
# Your solution here


In [ ]:
# Solution
query = '''
SELECT COUNT(*) AS total_orders
FROM orders
'''
pd.read_sql_query(query, conn)

### Exercise 3: Find products under $100

In [ ]:
# Your solution here


In [ ]:
# Solution
query = '''
SELECT product_name, category, price
FROM products
WHERE price < 100
ORDER BY price DESC
'''
pd.read_sql_query(query, conn)

## 🟡 Intermediate Exercises

### Exercise 4: Total revenue per customer

In [ ]:
# Your solution here


In [ ]:
# Solution
query = '''
SELECT 
    c.name,
    COUNT(o.order_id) AS order_count,
    SUM(o.total_amount) AS total_revenue
FROM customers c
LEFT JOIN orders o 
    ON c.customer_id = o.customer_id
    AND o.status = 'completed'
GROUP BY c.customer_id, c.name
ORDER BY total_revenue DESC
'''
pd.read_sql_query(query, conn)

### Exercise 5: Most popular product category

In [ ]:
# Your solution here


In [ ]:
# Solution
query = '''
SELECT 
    p.category,
    SUM(oi.quantity) AS total_sold
FROM order_items oi
JOIN products p 
    ON oi.product_id = p.product_id
GROUP BY p.category
ORDER BY total_sold DESC
'''
pd.read_sql_query(query, conn)

### Exercise 6: Customers with no orders

In [ ]:
# Your solution here


In [ ]:
# Solution
query = '''
SELECT 
    c.name,
    c.email,
    c.signup_date
FROM customers c
LEFT JOIN orders o 
    ON c.customer_id = o.customer_id
WHERE o.order_id IS NULL
'''
pd.read_sql_query(query, conn)

## 🔴 Advanced Exercises

### Exercise 7: Month-over-month revenue growth

In [ ]:
# Your solution here


In [ ]:
# Solution
query = '''
WITH monthly_revenue AS (
    SELECT 
        SUBSTR(order_date, 1, 7) AS month,
        SUM(total_amount) AS revenue
    FROM orders
    WHERE status = 'completed'
    GROUP BY month
)
SELECT 
    month,
    revenue,
    LAG(revenue) OVER (
        ORDER BY month
    ) AS prev_month_revenue,
    ROUND(
        100.0 * (revenue - LAG(revenue) OVER (
            ORDER BY month
        )) / LAG(revenue) OVER (
            ORDER BY month
        ), 
        2
    ) AS growth_pct
FROM monthly_revenue
ORDER BY month
'''
pd.read_sql_query(query, conn)

### Exercise 8: Top 3 products by revenue

In [ ]:
# Your solution here


In [ ]:
# Solution
query = '''
WITH product_revenue AS (
    SELECT 
        p.product_name,
        p.category,
        SUM(oi.quantity * p.price) AS revenue,
        RANK() OVER (
            ORDER BY SUM(oi.quantity * p.price) DESC
        ) AS rank
    FROM order_items oi
    JOIN products p 
        ON oi.product_id = p.product_id
    JOIN orders o 
        ON oi.order_id = o.order_id
    WHERE o.status = 'completed'
    GROUP BY p.product_id, p.product_name, 
             p.category
)
SELECT 
    product_name,
    category,
    revenue,
    rank
FROM product_revenue
WHERE rank <= 3
ORDER BY rank
'''
pd.read_sql_query(query, conn)

### Exercise 9: Customer lifetime value (CLV)

In [ ]:
# Your solution here


In [ ]:
# Solution
query = '''
WITH customer_metrics AS (
    SELECT 
        c.customer_id,
        c.name,
        c.signup_date,
        COUNT(o.order_id) AS order_count,
        SUM(CASE 
            WHEN o.status = 'completed' 
            THEN o.total_amount 
            ELSE 0 
        END) AS total_spent,
        AVG(CASE 
            WHEN o.status = 'completed' 
            THEN o.total_amount 
        END) AS avg_order_value
    FROM customers c
    LEFT JOIN orders o 
        ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.name, 
             c.signup_date
)
SELECT 
    name,
    order_count,
    ROUND(total_spent, 2) AS total_spent,
    ROUND(avg_order_value, 2) AS avg_order,
    RANK() OVER (
        ORDER BY total_spent DESC
    ) AS value_rank
FROM customer_metrics
WHERE total_spent > 0
ORDER BY total_spent DESC
'''
pd.read_sql_query(query, conn)

### Exercise 10: Product affinity analysis

In [ ]:
# Find products frequently bought together
# Your solution here


In [ ]:
# Solution
query = '''
SELECT 
    p1.product_name AS product_1,
    p2.product_name AS product_2,
    COUNT(*) AS times_bought_together
FROM order_items oi1
JOIN order_items oi2 
    ON oi1.order_id = oi2.order_id
    AND oi1.product_id < oi2.product_id
JOIN products p1 
    ON oi1.product_id = p1.product_id
JOIN products p2 
    ON oi2.product_id = p2.product_id
GROUP BY p1.product_id, p2.product_id,
         p1.product_name, p2.product_name
ORDER BY times_bought_together DESC
'''
pd.read_sql_query(query, conn)

## Bonus Challenges 🏆

### Challenge 1: RFM Analysis
Calculate Recency, Frequency, Monetary value 
for each customer.

In [ ]:
# Your solution here


### Challenge 2: Cohort Analysis
Group customers by signup month and track 
their purchase behavior.

In [ ]:
# Your solution here


In [ ]:
conn.close()

## Summary

You've practiced:
- ✅ Basic queries and filtering
- ✅ Aggregations and GROUP BY
- ✅ JOINs (INNER, LEFT)
- ✅ Window functions
- ✅ CTEs
- ✅ Real-world analytics

**Keep practicing!** 🚀